In [2]:
! pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.75.0
    Uninstalling openai-1.75.0:
      Successfully uninstalled openai-1.75.0


In [3]:
# Install required packages if not already installed
!pip install pandas requests streamlit --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 119.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00


In [4]:
# =============================
# 🛠 Step 1: Import Libraries
# =============================
import pandas as pd
import requests
import streamlit as st

In [7]:
# =============================
# 🛠 Step 2: Load the Data
# =============================
# Load the CSV file
df = pd.read_csv('/content/negative_comments_ocean.csv')  # Adjust path if needed

In [8]:
# =============================
# 🛠 Step 3: Prepare Prompt
# =============================
# Combine comments (limit to avoid token overflow)
all_comments = "\n".join(df['text'].astype(str).tolist())
all_comments = all_comments[:4000]  # Limit for safety

In [9]:
# Calculate average OCEAN scores
avg_ocean = {
    'openness': round(df['Openness'].mean(), 2),
    'conscientiousness': round(df['Conscientiousness'].mean(), 2),
    'extraversion': round(df['Extraversion'].mean(), 2),
    'agreeableness': round(df['Agreeableness'].mean(), 2),
    'neuroticism': round(df['Neuroticism'].mean(), 2),
}
dominant_trait = max(avg_ocean, key=avg_ocean.get)


In [10]:
# Create the prompt
prompt = f"""
You are an expert in psychology, public policy, and civic engagement.

You are given a batch of YouTube comments related to Indian democracy and elections.
These comments are mostly NEGATIVE in sentiment.

Here are some example comments:
\"\"\"
{all_comments}
\"\"\"

Here are the average personality scores for the commenters (OCEAN model):
- Openness: {avg_ocean['openness']}
- Conscientiousness: {avg_ocean['conscientiousness']}
- Extraversion: {avg_ocean['extraversion']}
- Agreeableness: {avg_ocean['agreeableness']}
- Neuroticism: {avg_ocean['neuroticism']}
Dominant Trait: {dominant_trait}

Please analyze and return a detailed response in this format:

Extracted Themes (main reasons behind the negative sentiment):
[List out 4–6 themes with short explanations in bullet form]

💬 Motivational Message:
[Write a warm, inspiring, personalized message that considers their personality and emotional state.]

✅ Civic Engagement Strategy:
[Offer an actionable and empathetic suggestion that helps this group turn their concern into productive democratic engagement.]

Make the language accessible, positive, and emotionally intelligent.
"""


In [ ]:
# =============================
# 🛠 Step 4: Send API Request
# =============================
# API request to OpenRouter
headers = {
    "Authorization": "Bearer ADD API KEY HERE",
    "Content-Type": "application/json"
}

data = {
    "model": "mistralai/mistral-7b-instruct",
    "messages": [{"role": "user", "content": prompt}],
    "temperature": 0.7
}

response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)

In [13]:
# Handle response
if response.status_code == 200:
    res_json = response.json()
    if "choices" in res_json:
        content = res_json["choices"][0]["message"]["content"]
        print("=== FULL ANALYSIS ===\n")
        print(content)

        # Save to text file
        with open("/content/detailed_response.txt", "w") as f:
            f.write(content)

        print("\n📁 Saved to: detailed_response.txt")
    else:
        print("Unexpected API response:\n", res_json)
else:
    print("Request failed:", response.status_code)
    print(response.text)

=== FULL ANALYSIS ===

 Extracted Themes (main reasons behind the negative sentiment):

1. Dissatisfaction with current political parties and leaders: Many comments express dissatisfaction with political parties, especially regional ones, for focusing on vested interests, engaging in false propaganda, and not solving local issues.
2. Frustration with chaos and corruption in Indian democracy: Several comments mention the chaos in elections and the belief that it leads to a chaotic government and country. They also express concern about politicians and their perceived corruption.
3. Concern about loss of accountability: Some comments worry that the consolidation of elections at the national level will result in a loss of accountability, as regional parties won't have to answer to their constituents as frequently.
4. Perception of bias in media coverage: A few comments accuse mainstream media of being biased and not being honest, which contributes to their negative sentiment towards the c

In [26]:
# =============================
# 🛠 Step 5: Build Streamlit App
# =============================
# Save this part separately as `streamlit_app.py`
streamlit_code = """
import streamlit as st

# Title
st.title("Democracy Comments Analysis")

# Load the detailed response
with open('/content/detailed_response.txt', 'r') as file:
    content = file.read()

# Display the content
st.header("📋 Detailed Analysis")
st.write(content)

"""

# Save the Streamlit app
with open('/content/streamlit_app.py', 'w') as f:
    f.write(streamlit_code)

print("\n✅ Streamlit app saved as 'streamlit_app.py'")


✅ Streamlit app saved as 'streamlit_app.py'


In [22]:

# =============================
# 🛠 Step 6: Run Streamlit App
# =============================

# In Colab, we need to use `localtunnel` or `ngrok` to expose Streamlit to the web
!pip install pyngrok --quiet

In [23]:
from pyngrok import ngrok
import threading


In [27]:
def run_streamlit():
    !streamlit run /content/streamlit_app.py --server.port 8501

# Run Streamlit in background
thread = threading.Thread(target=run_streamlit)
thread.start()

In [ ]:
from pyngrok import ngrok

# Set your auth token
ngrok.set_auth_token("ADD AUTH TOKEN HERE")

# Manually configure the tunnel
public_url = ngrok.connect(8501, "http")  # Explicitly define the protocol
print(f"\n🌐 Public URL to access Streamlit app: {public_url}")




🌐 Public URL to access Streamlit app: NgrokTunnel: "https://d031-34-145-67-170.ngrok-free.app" -> "http://localhost:8501"
